# Goal:

For the entries in the "job metrics" dataframe which have rinfo strings,
extract all the information from those rinfo strings and add that information to the 
dataframe in the form of new columns. 

This includes parsing data from the rinfo strings and converting to the appropriate format.

Then decide which columns to include in the training dataframe that will be passed to the
ML model. After rearranging and renaming those columns, save to a .pkl file.

# Imports and display settings

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 75


# Selecting features to extract from rinfo strings

The list of all features we will extract from the rinfo strings is contained in rinfo-features.txt.

In [2]:
features_filename = "rinfo-features.txt"
features_file = open(features_filename)
all_rinfo_features = features_file.read().split('\n')

print(all_rinfo_features)


['dates', 'dsnum', 'elon', 'enddate', 'format', 'grid_definition', 'gui', 'lats', 'level', 'lons', 'nlat', 'parameters', 'product', 'slat', 'startdate', 'station', 'tindex', 'wlon']


Some features are entered with slightly different syntax in different rinfo strings.

For example : grid definition may appear as either "grid_definition" or "grid-definition".

The next cell solves this problem by making a dictionary containing lists of alternate
names for each feature.

In [3]:
special_features = ['grid_definition', 'params', 'format']
normal_features = [feat for feat in all_rinfo_features
                   if feat not in special_features]

special_alt_names = {'grid_definition': ['grid_definition', 'grid-definition'],
                     'parameters': ['parameters', 'params', 'parms'],
                     'format': ['format', 'ofmt', 'fmt'],
                     'tindex': ['tindex', 'gindex']}

alternate_names = {feat: [feat] for feat in normal_features}
alternate_names.update(special_alt_names)


# Functions that parse rinfo strings

In [4]:
def handle_missing_rinfo_val(feature, val):
    """Decides the appropriate 'null value' for when a feature is
    not present in an rinfo string.
    """

    if feature == 'gui':
        return False
    elif feature in ['startdate', 'enddate']:
        return pd.NaT
    else:
        return float('nan')


In [5]:
def format_rinfo_val(rinfo, feature, val):
    """Formats a value found in an rinfo string so that it can be entered
    correctly into the pandas dataframe.
    
    Example input: 
    rinfo = '...;elon=76.4;...'
    feature = 'elon'
    val = '76.4'

    Example output: 76.4
    """

    if val == '':
        return handle_missing_rinfo_val(feature, val)

    try:
        if feature in ['elon', 'wlon', 'nlat', 'slat']:
            return float(val)
        elif feature in ['startdate', 'enddate']:
            return pd.to_datetime(val, errors='coerce')
        elif feature in ['gindex', 'tindex']:
            return float(val)
        elif feature == 'gui':
            return True if val == 'yes' else False
        else:
            return val
    except:
        failed_parse.write(f"Could not parse {feature} from {val}. \n")
        failed_parse.write(f"rinfo string: {rinfo} \n")
        return val


In [6]:
def get_val_from_rinfo(rinfo, feature):
    """Finds the value of the given feature in the given rinfo string.
    If the feature is not present in rinfo, calls handle_missing_rinfo_val.
    If the feature is present, uses format_rinfo_val to convert the value 
    to the appropriate data type before returning.
    
    Example input: 
    rinfo = '...;elon=76.4;...' 
    feature = 'elon'

    Example output: 76.4
    """
    
    rinfo = rinfo.replace('%3D', '=')
    
    if ';' in rinfo:
        sep = ';'
    else:
        sep = '&'

    val = ''
    for alternate_name in alternate_names[feature]:
        if rinfo.lower().find(alternate_name) != -1:
            start_ind = rinfo.lower().find(alternate_name) + len(alternate_name) + 1
            end_ind = rinfo.find(sep, start_ind)
            if end_ind != -1:
                val = rinfo[start_ind:end_ind]
            else:
                val = rinfo[start_ind:]
                
    val = format_rinfo_val(rinfo, feature, val)
    return val

# Parsing rinfo strings

In [7]:
def valid_rinfo(rinfo):
    """Decides whether an rinfo string is 'valid.'
    In practice, just serves to filter out a few problematic rinfo strings
    (14 out of the original 59803, or 0.023%).
    """
    if '\\n' in rinfo:
        return False
    elif '76,78,81,83,85,88,90,92,94,96grid_definition' in rinfo:
        return False
    elif 'startDate' in rinfo:
        return False
    else:
        return True


First read in the dataframe and filter out those which have missing or invalid rinfo strings.

In [28]:
df = pd.read_json('/glade/work/jdubeau/job-metrics-data.json')
df = df[df.rinfo.notnull()]
df['valid_rinfo'] = df.apply(lambda row: valid_rinfo(row['rinfo']), axis = 1)
df = df[df.valid_rinfo]

In [9]:
list(df.columns)

['start_date',
 'end_date',
 'status',
 'req_mem',
 'used_mem',
 'mem_delta',
 'percent_mem',
 'dsid',
 'request_type',
 'request_id',
 'partition_id',
 'rinfo',
 'cpus',
 'valid_rinfo']

Now add all the rinfo features as new columns. In case of any parsing errors, we
keep track of the rinfo strings that failed to parse in a file called failed-parse.txt.

In [10]:
failed_parse = open('failed-parse.txt', 'w')

for feature in all_rinfo_features:
    df[feature] = df.apply(lambda row: get_val_from_rinfo(row['rinfo'], feature), axis = 1)

failed_parse.close()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58546 entries, 6132011 to 6785820
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       58546 non-null  object        
 1   end_date         58546 non-null  object        
 2   status           58546 non-null  object        
 3   req_mem          58546 non-null  int64         
 4   used_mem         58546 non-null  float64       
 5   mem_delta        58546 non-null  float64       
 6   percent_mem      58546 non-null  float64       
 7   dsid             58546 non-null  object        
 8   request_type     58546 non-null  object        
 9   request_id       58546 non-null  float64       
 10  partition_id     45037 non-null  float64       
 11  rinfo            58546 non-null  object        
 12  cpus             55774 non-null  float64       
 13  valid_rinfo      58546 non-null  bool          
 14  dates            12691 non-nul

# Combining redundant columns

The first redundancy we deal with are the columns called "lats" and "lons." 
A "lats" entry, for example, would look like "60 S 80 N." This information should
be split up and entered into the "slat" and "nlat" columns in the same row, by 
setting slat = -60.0 and nlat = 80.0.

In [12]:
def parse_lats_lons(val):
    """Takes a 'lats' or 'lons' value and returns two floats representing the 
    southern/western coordinate and the northern/eastern coordinate.
    Example input: '60 S 80 N'
    Example output: (-60.0, 80.0)
    """
    val = val.replace(',', '')
    substrings = val.split(' ')

    try:
        first_coord = float(substrings[0])
        second_coord = float(substrings[2])
    except:
        print(f"Error expanding lats/lons. Value = {val}")
        return (float('nan'), float('nan'))

    if substrings[1] == 'W' or substrings[1] == 'S':
        first_coord = -1*first_coord
    if substrings[3] == 'W' or substrings[3] == 'S':
        second_coord = -1*second_coord

    return (first_coord, second_coord)


In [13]:
def update_lat_lon(feature, row):
    """Used to update 'slat', 'nlat', 'wlon', or 'elon' by 
    getting the values from 'lats' or 'lons' in the same row.
    Example input: ('nlat', <row containing 'lats=45 S 50 N'>)
    Example output: 50.0
    """
    if row['lats'] != row['lats']:
        return row[feature]
    else:
        if feature == 'slat':
            return parse_lats_lons(row['lats'])[0]
        elif feature == 'nlat':
            return parse_lats_lons(row['lats'])[1]
        elif feature == 'wlon':
            return parse_lats_lons(row['lons'])[0]
        else:
            return parse_lats_lons(row['lons'])[1]


Compare the df.info() output with the previous df.info(); there should be more non-null slat's, nlat's, etc.

In [14]:
for feature in ['slat', 'nlat', 'wlon', 'elon']:
    df[feature] = df.apply(lambda row: update_lat_lon(feature, row), axis = 1)

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58546 entries, 6132011 to 6785820
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       58546 non-null  object        
 1   end_date         58546 non-null  object        
 2   status           58546 non-null  object        
 3   req_mem          58546 non-null  int64         
 4   used_mem         58546 non-null  float64       
 5   mem_delta        58546 non-null  float64       
 6   percent_mem      58546 non-null  float64       
 7   dsid             58546 non-null  object        
 8   request_type     58546 non-null  object        
 9   request_id       58546 non-null  float64       
 10  partition_id     45037 non-null  float64       
 11  rinfo            58546 non-null  object        
 12  cpus             55774 non-null  float64       
 13  valid_rinfo      58546 non-null  bool          
 14  dates            12691 non-nul

Next we deal with the 'dates' column. Here there are three possibilities:

1. The 'dates' field contains two dates, representing a start and end date, 
e.g. "dates=2019-01-01 00:00 2019-12-31 18:00"

2. The 'dates' field says the dates were initial (as opposed to valid),
e.g. "dates=init"

3. The 'dates' field was empty, indicating that the dates were valid.


In [15]:
def parse_dates(feature, dates):
    """Deduces a start date or end date from whatever was in
    the 'dates' column. The entered feature must be either 
    'startdate' or 'enddate'.
    Example input: ('enddate', '2019-01-01 00:00 2019-12-31 18:00')
    Example output: 2019-12-31 18:00 (pandas datetime object)
    """
    dates_split = dates.split(' ')
        
    if len(dates_split) == 4:
        # Typical case: dates=2019-01-01 00:00 2019-12-31 18:00
        if feature == 'startdate':
            date = dates_split[0] + ' ' + dates_split[1]
        else:
            date = dates_split[2] + ' ' + dates_split[3]
    else:
        # Typical cases: either dates=1806-01-01 1900-12-31
        # or dates=197005 201412
        if feature == 'startdate':
            date = dates_split[0]
        else:
            date = dates_split[1]
            
        if '-' not in dates_split[0]:
            date = date[:4] + '-' + date[4:]
                
    return pd.to_datetime(date, errors='coerce')


In [16]:
def update_dates(feature, row):
    """Used to update 'startdate', 'enddate', or 'dates_init' by
    getting the values from the 'dates' column in the same row.
    feature must be either 'startdate', 'enddate', or 'dates_init'.
    """
    dates = row['dates']
    
    if feature == 'dates_init':
        return True if dates == 'init' else False

    if row[feature] == row[feature]:
        return row[feature]
    else:
        if row['dates'] == row['dates'] and row['dates'] != 'init':
            return parse_dates(feature, dates)
        else:
            return pd.NaT


Again, after we combine the info from 'dates', there should be more
non-null 'startdate's and 'enddates' compared to the previous df.info() call.

In [17]:
date_features = ['dates_init', 'startdate', 'enddate']
for feature in date_features:
    df[feature] = df.apply(lambda row: update_dates(feature, row), axis = 1)
    
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58546 entries, 6132011 to 6785820
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       58546 non-null  object        
 1   end_date         58546 non-null  object        
 2   status           58546 non-null  object        
 3   req_mem          58546 non-null  int64         
 4   used_mem         58546 non-null  float64       
 5   mem_delta        58546 non-null  float64       
 6   percent_mem      58546 non-null  float64       
 7   dsid             58546 non-null  object        
 8   request_type     58546 non-null  object        
 9   request_id       58546 non-null  float64       
 10  partition_id     45037 non-null  float64       
 11  rinfo            58546 non-null  object        
 12  cpus             55774 non-null  float64       
 13  valid_rinfo      58546 non-null  bool          
 14  dates            12691 non-nul

# Final cleaning and saving

Some of the SLURM jobs have reported using zero memory. We consider these
entries to be untrustworthy so we remove them from the dataset.

In [18]:
df = df[df.used_mem > 0]

There was a scaling issue in converting SLURM's requested and used memory numbers into megabytes. SLURM reports requested memory in 1024-scaled gigabytes, which we incorrectly converted to megabytes by multiplying by 1000. For the used memory, SLURM reports the number in kilobytes, which we incorrectly converted to megabytes by dividing by 1000. 

In [19]:
df['req_mem_adj'] = df.apply(lambda row: row['req_mem'] * 1.024,
                             axis = 1)
df['used_mem_adj'] = df.apply(lambda row: row['used_mem'] * 0.9765625,
                             axis = 1)


At the moment, 'start_date' refers to the date/time that the SLURM job began, whereas 'startdate' refers to the date/time that the requested data starts from. We rename those two to avoid confusion. We do the same for 'end_date' and 'enddate.' 

Then we rename used_mem_adj and req_mem_adj to simply be 
used_mem and req_mem, keeping the original used_mem and req_mem as 
used_mem_old and req_mem_old.

In [20]:
df = df.rename(columns={'start_date': 'job_start_at',
                        'end_date': 'job_end_at',
                        'enddate': 'rqst_end_at',
                        'startdate': 'rqst_start_at',
                        'used_mem': 'used_mem_old',
                        'req_mem': 'req_mem_old',
                        'used_mem_adj': 'used_mem',
                        'req_mem_adj': 'req_mem'})


In [21]:
list(df.columns)

['job_start_at',
 'job_end_at',
 'status',
 'req_mem_old',
 'used_mem_old',
 'mem_delta',
 'percent_mem',
 'dsid',
 'request_type',
 'request_id',
 'partition_id',
 'rinfo',
 'cpus',
 'valid_rinfo',
 'dates',
 'dsnum',
 'elon',
 'rqst_end_at',
 'format',
 'grid_definition',
 'gui',
 'lats',
 'level',
 'lons',
 'nlat',
 'parameters',
 'product',
 'slat',
 'rqst_start_at',
 'station',
 'tindex',
 'wlon',
 'dates_init',
 'req_mem',
 'used_mem']

In [22]:
training_features = ['request_type', 'job_start_at',
                     'job_end_at', 'req_mem', 'used_mem',
                     'mem_delta', 'percent_mem',
                     'dsnum', 'rqst_start_at', 'rqst_end_at',
                     'dates_init', 'slat', 'nlat',
                     'wlon', 'elon', 'gui', 'parameters',
                     'grid_definition', 'level', 'product',
                     'station', 'tindex', 'format'] 

training_df = df[training_features]


In [23]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50360 entries, 6132011 to 6785810
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   request_type     50360 non-null  object        
 1   job_start_at     50360 non-null  object        
 2   job_end_at       50360 non-null  object        
 3   req_mem          50360 non-null  float64       
 4   used_mem         50360 non-null  float64       
 5   mem_delta        50360 non-null  float64       
 6   percent_mem      50360 non-null  float64       
 7   dsnum            48150 non-null  object        
 8   rqst_start_at    50327 non-null  datetime64[ns]
 9   rqst_end_at      50248 non-null  datetime64[ns]
 10  dates_init       50360 non-null  bool          
 11  slat             32123 non-null  float64       
 12  nlat             32123 non-null  float64       
 13  wlon             32140 non-null  float64       
 14  elon             32140 non-nul

In [24]:
training_path = '/glade/work/jdubeau/job-metrics-training.json'
training_df.to_json(training_path, date_format = 'iso')


In [25]:
test_df = pd.read_json(training_path, dtype={'dsnum':'object'})
test_df_2 = pd.read_json(training_path)

In [27]:
test_df.sample(10)

,request_type,job_start_at,job_end_at,req_mem,used_mem,mem_delta,percent_mem,dsnum,rqst_start_at,rqst_end_at,dates_init,slat,nlat,wlon,elon,gui,parameters,grid_definition,level,product,station,tindex,format
6362354,PP,2021-01-06 09:59:11,2021-01-06 10:28:40,4096.0,38.839844,3960.228,0.994300,627.0,1979-01-01 00:00:00+00:00,1989-01-01 23:00:00+00:00,False,NaN,NaN,NaN,NaN,True,"1!98-0.128:205,1!98-0.128:228,1!98-0.128:39,1!...",None,None,None,None,NaN,None
6532407,PP,2021-02-22 20:21:15,2021-02-22 20:22:14,2048.0,37.511719,1961.588,1.920600,084.1,2019-01-01 00:00:00+00:00,2020-12-31 12:00:00+00:00,False,29.00000,30.00000,119.000000,120.00000,True,3!7-0.2-1:0.0.0,None,221,32,None,NaN,None
6321440,SP,2020-12-23 04:02:08,2020-12-23 04:03:18,1024.0,76.136719,922.036,7.796400,628.1,1958-01-01 00:00:00+00:00,2019-12-31 18:00:00+00:00,False,NaN,NaN,NaN,NaN,True,1!34-241.200:11,None,None,None,None,25.0,netCDF
6616487,PP,2021-03-08 06:17:13,2021-03-08 06:21:53,2048.0,40.996094,1958.020,2.099000,084.1,2020-10-01 00:00:00+00:00,2020-10-31 12:00:00+00:00,True,24.00000,50.00000,-125.000000,-66.00000,False,3!7-0.2-1:0.0.4,None,221,None,None,NaN,None
6425610,PP,2021-01-23 02:54:05,2021-01-23 02:55:01,1024.0,35.269531,963.884,3.611600,631.1,2000-01-01 00:00:00+00:00,2016-12-31 21:00:00+00:00,False,NaN,NaN,NaN,NaN,True,"8!ds631.1:SH2O,8!ds631.1:SMOIS,8!ds631.1:TSLB,...",None,None,None,None,5.0,None
6410205,PP,2021-01-20 02:56:04,2021-01-20 03:05:55,2048.0,62.191406,1936.316,3.184200,084.1,2019-01-01 00:00:00+00:00,2019-06-01 00:00:00+00:00,True,15.00000,60.00000,70.000000,140.00000,True,"3!7-0.2-1:0.4.192,3!7-0.2-1:0.3.0,3!7-0.2-1:0....",None,"107,76,79,81,84,86,221,223,1308,1309","1577,139,1581,666,26,1592,668,144,1600,670,29,...",None,NaN,None
6368817,PP,2021-01-08 16:06:03,2021-01-08 19:30:29,2048.0,164.085938,1831.976,8.401200,084.1,2016-12-01 00:00:00+00:00,2019-01-31 12:00:00+00:00,False,15.00000,56.00000,71.000000,139.00000,True,3!7-0.2-1:0.3.0,None,107,"119,3,124,23,129,41,134,32,139,26,144,29,149,3...",None,NaN,netCDF
6183070,PP,2020-11-11 17:07:14,2020-11-11 17:11:16,8192.0,31.335938,7967.912,0.401100,None,2012-01-01 00:00:00+00:00,2012-12-31 00:00:00+00:00,False,-46.00000,-45.00000,-68.000000,-67.00000,False,PRES PMSL TMDB ALSE TMDP REHU WDIR WSPD TP03 T...,None,None,None,None,NaN,None
6337201,PP,2020-12-30 02:19:19,2020-12-30 02:20:11,8192.0,49.191406,7949.628,0.629650,None,2017-11-01 00:00:00+00:00,2017-12-28 00:00:00+00:00,False,0.00000,90.00000,100.000000,180.00000,False,WDIR WSPD,None,None,None,None,NaN,None
6490078,PP,2021-02-09 07:13:46,2021-02-09 07:21:40,51200.0,58.414062,49940.184,0.119632,084.1,2016-06-01 00:00:00+00:00,2021-02-09 00:00:00+00:00,True,40.42186,55.72711,46.318359,87.51709,False,3!7-0.2-1:0.0.0,None,221,"41,119,1,3",None,NaN,None
